In [0]:
from datetime import date, timedelta
start_date = date.today() - timedelta(1)

silver_adls = "abfss://silver@earthquakedatalake1.dfs.core.windows.net/"
gold_adls = "abfss://gold@earthquakedatalake1.dfs.core.windows.net/"

silver_data = f"{silver_adls}earthquake_events_silver/"

In [0]:
%pip install reverse_geocoder

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import StringType
import reverse_geocoder as rg
from datetime import date, timedelta

In [0]:
df = spark.read.parquet(silver_data).filter(col('time') > start_date)

In [0]:
%python
spark.conf.set(
    "fs.azure.account.key.earthquakedatalake1.dfs.core.windows.net",
    "SRUji+k7sVYXBvlYcvPcWy6KfqrcpTGX0UF2vjyrSCWrkrmU1sALPnnoBY8Hctqo2ZDOl+TSLIyc+AStq+eIvA=="
)

df = spark.read.parquet("abfss://silver@earthquakedatalake1.dfs.core.windows.net/earthquake_events_silver/").filter(col('time') > start_date)
display(df)

id,longitude,latitude,elevation,title,place_description,sig,mag,magType,time,updated
ci40930655,-117.9233333,35.3216667,7.05,"M 2.2 - 22 km NNE of California City, CA","22 km NNE of California City, CA",76,2.22,ml,2025-04-16T23:55:22.17Z,2025-04-16T23:58:49.679Z
uw62092951,-117.887,47.35916666666667,-0.61,"M 2.4 Explosion - 9 km NE of Sprague, Washington","9 km NE of Sprague, Washington",91,2.43,ml,2025-04-16T23:53:55.02Z,2025-04-17T00:19:14.44Z
ci40930639,-116.4801667,33.137,13.69,"M 1.2 - 13 km ENE of Julian, CA","13 km ENE of Julian, CA",21,1.16,ml,2025-04-16T23:36:11.71Z,2025-04-16T23:40:09.187Z
ci40930631,-116.6105,33.056,13.2,"M 1.4 - 3 km SSW of Julian, CA","3 km SSW of Julian, CA",30,1.4,ml,2025-04-16T23:35:46.75Z,2025-04-16T23:39:24.181Z
ci40930615,-116.6441667,33.0926667,14.34,"M 1.2 - 4 km WNW of Julian, CA","4 km WNW of Julian, CA",23,1.22,ml,2025-04-16T23:29:42.95Z,2025-04-16T23:33:30.14Z
tx2025hlqa,-104.001,31.516,4.5358,"M 1.5 - 29 km NW of Toyah, Texas","29 km NW of Toyah, Texas",35,1.5,ml,2025-04-16T23:19:55.021Z,2025-04-16T23:24:30.226Z
ak0254vora5z,-147.4797,64.9874,0.1,"M 1.1 - 7 km ENE of Fox, Alaska","7 km ENE of Fox, Alaska",19,1.1,ml,2025-04-16T23:16:44.717Z,2025-04-16T23:18:44.016Z
ci40930583,-117.8688333,33.8961667,8.77,"M 1.2 - 3 km N of Placentia, CA","3 km N of Placentia, CA",21,1.18,ml,2025-04-16T22:51:18.33Z,2025-04-16T22:55:04.902Z
ak0254vob6i5,-149.8275,62.6685,63.0,"M 2.2 - 28 km NNE of Chase, Alaska","28 km NNE of Chase, Alaska",74,2.2,ml,2025-04-16T22:41:32.291Z,2025-04-16T22:43:26.742Z
ci40930575,-117.7775,36.0291667,1.43,"M 1.5 - 16 km E of Coso Junction, CA","16 km E of Coso Junction, CA",34,1.49,ml,2025-04-16T22:40:35.66Z,2025-04-16T22:44:07.819Z


In [0]:

df = df.limit(100)

In [0]:
def get_country_code(lat, lon):
    """
    Retrieve the country code for a given latitude and longitude.

    Parameters:
    lat (float or str): Latitude of the location.
    lon (float or str): Longitude of the location.

    Returns:
    str: Country code of the location, retrieved using the reverse geocoding API.

    Example:
    >>> get_country_details(48.8588443, 2.2943506)
    'FR'
    """
    try:
        coordinates = (float(lat), float(lon))
        result = rg.search(coordinates)[0].get('cc')
        print(f"Processed coordinates: {coordinates} -> {result}")
        return result
    except Exception as e:
        print(f"Error processing coordinates: {lat}, {lon} -> {str(e)}")
        return None

In [0]:
get_country_code_udf = udf(get_country_code, StringType())

In [0]:

get_country_code(48.8588443, 2.2943506)

Loading formatted geocoded file...
Processed coordinates: (48.8588443, 2.2943506) -> FR


'FR'

In [0]:
df_with_location = \
                df.\
                    withColumn("country_code", get_country_code_udf(col("latitude"), col("longitude")))

In [0]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- elevation: double (nullable = true)
 |-- title: string (nullable = true)
 |-- place_description: string (nullable = true)
 |-- sig: long (nullable = true)
 |-- mag: double (nullable = true)
 |-- magType: string (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- updated: timestamp (nullable = true)



In [0]:

df_with_location.printSchema()

root
 |-- id: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- elevation: double (nullable = true)
 |-- title: string (nullable = true)
 |-- place_description: string (nullable = true)
 |-- sig: long (nullable = true)
 |-- mag: double (nullable = true)
 |-- magType: string (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- updated: timestamp (nullable = true)
 |-- country_code: string (nullable = true)



In [0]:
df_with_location_sig_class = \
                            df_with_location.\
                                withColumn('sig_class', 
                                            when(col("sig") < 100, "Low").\
                                            when((col("sig") >= 100) & (col("sig") < 500), "Moderate").\
                                            otherwise("High")
                                            )

In [0]:
df_with_location_sig_class.printSchema()

root
 |-- id: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- elevation: double (nullable = true)
 |-- title: string (nullable = true)
 |-- place_description: string (nullable = true)
 |-- sig: long (nullable = true)
 |-- mag: double (nullable = true)
 |-- magType: string (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- updated: timestamp (nullable = true)
 |-- country_code: string (nullable = true)
 |-- sig_class: string (nullable = false)



In [0]:
gold_output_path = f"{gold_adls}earthquake_events_gold/"

In [0]:

df_with_location_sig_class.write.mode('append').parquet(gold_output_path)

In [0]:
spark.conf.set(
    "fs.azure.account.key.earthquakedatalake1.dfs.core.windows.net",
    "SRUji+k7sVYXBvlYcvPcWy6KfqrcpTGX0UF2vjyrSCWrkrmU1sALPnnoBY8Hctqo2ZDOl+TSLIyc+AStq+eIvA=="
)

df_with_location_sig_class.write.mode('append').parquet(gold_output_path)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-5505944329879004>, line 1
----> 1 df = spark.read.parquet(silver_data).filter(col('time') > start_date)

NameError: name 'silver_data' is not defined